In [0]:
layer = "gold"
nm_gold_total = "kpi_total"

In [0]:
%run ../functions

In [0]:
df_total = spark.sql("""

with percent as (
  select
    percentile_approx(resp_time, 0.25) as q1,
    percentile_approx(resp_time, 0.75) as q3,
    percentile_approx(resp_time, 0.75)
      + 1.5 * (
        percentile_approx(resp_time, 0.75)
        - percentile_approx(resp_time, 0.25)
      ) as percent_limit
  from osticket.silver.thread_entry_agg
),

filtered_base as (
  select
    t.resp_time,
    year(created) as year,
    month(created) as month,
    (year(created) * 100 + month(created)) as year_month
  from osticket.silver.thread_entry_agg t
  cross join percent p
  where t.resp_time <= p.percent_limit
  and t.id_staff is not null
)

select
  year_month,
  year,
  month,
  median(resp_time) as median_time,
  floor(median_time/60) as hours,
  floor(mod(median_time, 60)) as minutes,
  concat(
  lpad(cast(hours as string), 2, '0'),
  ':',
  lpad(cast(minutes as string), 2, '0')
) as hours_minutes
from filtered_base
group by year, month, year_month
order by year, month, year_month                
""")
df_total.display()


Decisão de uso da mediana e IQR:
-
-
Aqui foi decidio o uso de mediana ao uso de média, como o tempo de resposta em alguns chamados pode alterar de forma muito significativa por n motivos, assim pegaremos um valor central do tempo de resposta. Pra finalizar e dar um contexto mais real do tempo de resposta de cada staff, resolvi usar o método IQR para remover da contagem valores muito descrepantes, mesmo que a mediana já desse uma valor central, removendo esses outliers temos um valor mais acertivo para o nosso contexto.


In [0]:
write_table(df_total, layer, nm_gold_total)